## Prelim Scheduling

In a previous lab you had to come up with ideas for a model that solves the problem of scheduling prelims to day,time and rooms. Now you will be given a slightly modified version of the model that was used to schedule Cornell Spring 2021 prelims, complete the code that is provided and solve the model using Google OR-Tools.

In [1]:
# imports the modules we use throughout the notebook
import numpy as np
import pandas as pd
import itertools
from ortools.linear_solver import pywraplp

input_data_path = './Data/'

We first go over the data that we will be using(the cells that go over the data are identical to the ones used in the previous lab, as the data is the same). Courses can request to have prelims, and can express their preferences as to when the exams will be scheduled by giving $3$ prefered dates on which they would like to have the exam. For simplicity, we assume that all $3$ dates are always present. 
<br> <br>
Formally, we have a set of prelim exams, $I=\{1,\ldots,M\}$. Each prelim  $i\in I$ has
- A unique exam id, which will be denoted by $i \in I$.
- A class name, which is the name of the course associated with the prelim.
- The academic organization(ie CS, ORIE, MATH), that the class belongs to.
-  Enrollment size $s_i\in S \in \mathbb{N}$, which is the number of students that have enrolled in the class.
-  The modality of the prelim, which can be Online or In person. 
-  A 1st, 2nd and 3rd preferred date. We want to have the exam on one of those dates, in that order of preference. Let us denote them as $p_{i,n}, $ for $i \in I$ and $n = 1,2,3$ and $P_i = \{p_{i,1},p_{i,2},p_{i,3}\}$.

In the cell bellow, we read the file containing all of the prelims to be scheduled, along with the information described above. 

In [3]:
# df with prelim exams requested
exams = (pd.read_csv('prelim_exams.csv', index_col = 'exam_id'))
exams.loc[exams.modality == 'Online','enrollment'] = 0
exams

,course,acadorg,enrollment,modality,prefdate,prefdate2,prefdate3
exam_id,,,,,,,
1-AEM-2210-LEC-2634167-1,AEM 2210,AEM,0,Online,2021-03-18,2021-03-16,2021-03-23
1-AEM-2210-LEC-2634167-2,AEM 2210,AEM,0,Online,2021-04-15,2021-04-13,2021-04-20
1-AEM-2225-LEC-2634167-1,AEM 2225,AEM,0,Online,2021-03-23,2021-03-18,2021-03-25
1-AEM-2225-LEC-2634167-2,AEM 2225,AEM,0,Online,2021-04-20,2021-04-15,2021-04-22
1-AEM-2240-LEC-3778494-1,AEM 2240,AEM,270,In person,2021-03-16,2021-03-04,2021-03-18
...,...,...,...,...,...,...,...
1-STSCI-1380-LEC-1757307-1,STSCI 1380,STSCI,64,In person,2021-03-16,2021-03-18,2021-03-04
1-STSCI-1380-LEC-1757307-2,STSCI 1380,STSCI,64,In person,2021-04-20,2021-04-22,2021-04-15
1-STSCI-2150-LEC-1319792-1,STSCI 2150,STSCI,140,In person,2021-03-18,2021-03-16,2021-03-23


We are given a set of days on which the exams have to be scheduled. Every day has $K$  different times that an exam can start on. 
So let $D$ be the set of days of the semester on which we will schedule prelims, and $K$ be the set of starting times for exams on each day.
<br><br>
You can assume that the prefered days that the courses gave in the data always fall in $D$.

In [4]:
# reads the available exam dates
exam_dates = (pd.read_csv('avail_prel_dates.csv')).exam_dates.tolist()
# number of slots per day
K = 2
exam_dates

['2021-02-25',
 '2021-03-02',
 '2021-03-04',
 '2021-03-16',
 '2021-03-18',
 '2021-03-23',
 '2021-03-25',
 '2021-03-30',
 '2021-04-01',
 '2021-04-06',
 '2021-04-08',
 '2021-04-13',
 '2021-04-15',
 '2021-04-20',
 '2021-04-22',
 '2021-04-29',
 '2021-05-04',
 '2021-05-06',
 '2021-05-11',
 '2021-05-13']

For the exams that will take place in person, we are given a set of rooms $N$, where we can schedule the exams. Each room $r\in N$, has capacity $s_r$, which is the numbers of seats available in that room, accounting for some empty seats due to Covid restriction. There is also a file containing the distance between buildings, so in the case where a prelim is assigned multiple rooms, we can try to limit how spread out they are.

In [5]:
# reads the room and building dist dfs
rooms = (pd.read_csv('rooms.csv', index_col = 'room_id'))
building_dist = (pd.read_csv('buildings_dist.csv', index_col = 0))
display(rooms)
display(building_dist.head(10))

,capacity,building
room_id,,
Morrison Hall-342,9,Morrison Hall
Physical Sciences Building-401,9,Physical Sciences Building
Rockefeller Hall-102,8,Rockefeller Hall
Olin Hall-128,9,Olin Hall
Baker Laboratory-G02,8,Baker Laboratory
...,...,...
Sibley Hall-235,65,Sibley Hall
Statler Hall Auditorium-185,76,Statler Hall Auditorium
Schwartz Ctr Performing Arts-111,78,Schwartz Ctr Performing Arts


,Morrison Hall,Physical Sciences Building,Rockefeller Hall,Olin Hall,Baker Laboratory,White Hall,Weill Hall,Riley-Robb Hall,Plant Science Building,Warren Hall,...,Kennedy Hall,Milstein Hall,Phillips Hall,Biotechnology,Klarman Hall,Uris Library,Anabel Taylor Hall,Sibley Hall,Schwartz Ctr Performing Arts,Bailey Hall
Morrison Hall,0.000000,0.684845,0.664750,0.773209,0.692658,0.867034,0.420573,0.102957,0.416633,0.445776,...,0.525352,0.804689,0.660057,0.464509,0.725549,0.828247,0.837567,0.819629,0.889184,0.586574
Physical Sciences Building,0.684845,0.000000,0.062251,0.338360,0.020750,0.185802,0.302028,0.610824,0.268815,0.244710,...,0.178271,0.128547,0.373643,0.297855,0.082539,0.234214,0.396508,0.136246,0.559373,0.099358
Rockefeller Hall,0.664750,0.062251,0.000000,0.282155,0.083001,0.204554,0.263177,0.584533,0.249718,0.240273,...,0.141969,0.177748,0.311421,0.249907,0.061634,0.201613,0.343565,0.176209,0.502183,0.087859
Olin Hall,0.773209,0.338360,0.282155,0.000000,0.357541,0.342742,0.367137,0.674295,0.427648,0.455960,...,0.320362,0.403243,0.137407,0.313952,0.265707,0.173671,0.070338,0.380729,0.221176,0.342415
Baker Laboratory,0.692658,0.020750,0.083001,0.357541,0.000000,0.183860,0.316642,0.620729,0.278001,0.249645,...,0.193334,0.114992,0.394386,0.314952,0.097671,0.247618,0.414730,0.126976,0.578668,0.110959
White Hall,0.867034,0.185802,0.204554,0.342742,0.183860,0.000000,0.466767,0.788748,0.450476,0.430382,...,0.346524,0.110969,0.437094,0.447665,0.147961,0.172918,0.373643,0.078257,0.547018,0.280990
Weill Hall,0.420573,0.302028,0.263177,0.367137,0.316642,0.466767,0.000000,0.330426,0.100141,0.160390,...,0.123811,0.430561,0.283389,0.057498,0.319300,0.407981,0.435951,0.435903,0.529809,0.207164
Riley-Robb Hall,0.102957,0.610824,0.584533,0.674295,0.620729,0.788748,0.330426,0.000000,0.342828,0.382075,...,0.442967,0.734863,0.558520,0.369820,0.644056,0.737837,0.737609,0.746932,0.786296,0.511507
Plant Science Building,0.416633,0.268815,0.249718,0.427648,0.278001,0.450476,0.100141,0.342828,0.000000,0.062460,...,0.118199,0.392041,0.366344,0.149587,0.311183,0.430111,0.497965,0.404531,0.610643,0.170022
Warren Hall,0.445776,0.244710,0.240273,0.455960,0.249645,0.430382,0.160390,0.382075,0.062460,0.000000,...,0.135632,0.359423,0.410154,0.204719,0.301198,0.434538,0.525991,0.376107,0.650349,0.153111


Finally, we are given the coenrollment matrix. This has one entry for each one of combination of prelims that have a coenrollment conflict, meaning that there is some number of students enrolled in both courses corresponding to the exams. Duplicates and conflicts that an exam has with itself have been dropped.

In [6]:
# read the coenrollment df
coenrollments = (pd.read_csv('coenrollment_s21_prelims.csv'))
# only keep the exams where there are students in common
coenrollments = coenrollments[coenrollments.coenrollment != 0]
coenrollments

,exam_id_1,course_1,exam_id_2,course_2,coenrollment
0,1-AEM-2210-LEC-2634167-1,AEM 2210,1-AEM-2241-LEC-1001120-1,AEM 2241,5
1,1-AEM-2210-LEC-2634167-1,AEM 2210,1-AEM-2241-LEC-1001120-2,AEM 2241,5
2,1-AEM-2210-LEC-2634167-1,AEM 2210,1-AEM-2241-LEC-1001120-3,AEM 2241,5
3,1-AEM-2210-LEC-2634167-2,AEM 2210,1-AEM-2241-LEC-1001120-1,AEM 2241,5
4,1-AEM-2210-LEC-2634167-2,AEM 2210,1-AEM-2241-LEC-1001120-2,AEM 2241,5
...,...,...,...,...,...
5826,1-PHYS-2214-LEC-1017932-1,PHYS 2214,1-PHYS-2217-LEC-1009483-2,PHYS 2217,1
5827,1-PHYS-2214-LEC-1017932-2,PHYS 2214,1-PHYS-2217-LEC-1009483-1,PHYS 2217,1
5828,1-PHYS-2214-LEC-1017932-2,PHYS 2214,1-PHYS-2217-LEC-1009483-2,PHYS 2217,1
5829,1-PHYS-3318-LEC-2319573-1,PHYS 3318,1-PHYS-4443-LEC-1013508-1,PHYS 4443,10


## Variables

We are now ready to start defining our model. We first create binary variables $y$ to indicate whether a prelim has been assigned to a specific day/time. So we want to have binary $y(i,d,k)$ for all $i \in I, d \in P_i, k \in K$. Notice we only create the variables $y$ for days that are in the three prefered days of the exam $i$.

In [7]:
m = pywraplp.Solver.CreateSolver('SCIP_MIXED_INTEGER_PROGRAMMING')

Before you start creating the variables, in the cell bellow you can find some pythons expressions that you might find usefull in creating the sets of indices you need.

In [8]:
# the set of the prelims to schedule are the indices of the dataframe
I = exams.index.tolist()
display(I[0:20])
i = I[5]
# this is one of the ways to get the prefered days of i
print(exams.loc[i, ['prefdate','prefdate2','prefdate3']].tolist())

['1-AEM-2210-LEC-2634167-1',
 '1-AEM-2210-LEC-2634167-2',
 '1-AEM-2225-LEC-2634167-1',
 '1-AEM-2225-LEC-2634167-2',
 '1-AEM-2240-LEC-3778494-1',
 '1-AEM-2240-LEC-3778494-2',
 '1-AEM-2241-LEC-1001120-1',
 '1-AEM-2241-LEC-1001120-2',
 '1-AEM-2241-LEC-1001120-3',
 '1-AEM-2300-LEC-1000205-1',
 '1-AEM-2300-LEC-1000205-2',
 '1-AEM-2770-LEC-1023161-1',
 '1-AEM-3100-LEC-1008805-1',
 '1-AEM-3230-LEC-2041935-1',
 '1-AEM-3230-LEC-2041935-2',
 '1-AEM-3370-LEC-3225389-1',
 '1-AEM-3370-LEC-3225389-2',
 '1-AEM-3370-LEC-3225389-3',
 '1-AEM-4280-LEC-1281313-1',
 '1-AEM-4280-LEC-1281313-2']

['2021-04-13', '2021-04-08', '2021-04-15']


In [9]:
# defining the y variables
# remember we only have 2 slots per day, so K is set to 2
K = 2
y =  {(i,d,k):m.IntVar(0, 1, 'y'+str([i,d,k])) 
      for i in I for d in exams.loc[i, ['prefdate','prefdate2','prefdate3']].tolist() 
      for k in range(K)}

Next, since we want to assign one or more rooms to each exam, we would want to create binary variables to indicate whether a room is selected for the exam or not. We would create one variable $x(i,r,d,k)$ for each $i \in I, r \in N, d \in P_i, k \in K$. Use the cell bellow to count and print how many of the $x$ variables we would need to create.

In [10]:
num_new_vars = len(exams.index)*len(rooms.index)*3*2
num_new_vars

124200

You probably found out that we would need to create a very large number of $x$ variables. Since in the next steps, we will be using those variables for constraints, we would end up with a very large model. In order to bypass that, instead of using the actual rooms, we will use room buckets, which will be read from a file in the next cell. Suppose the set of room buckets is $R$ and $rb \in R$ is a room bucket. For every $rb$, there is a quantity $n_{rb}$, which will denote the number of rooms that are in that room bucket. This means that we have $n_{rb}$ rooms of size $rb$ available. You might notice that we create a room bucket of size zero, and number of rooms equal to the number of exams. That is done so we can assign all the online exams to that bucket, and that is why we set the enrollments to zero. So now instead of assigning exams to rooms, we will assign exams to room buckets, which act as a proxy for rooms.

In [13]:
# read the room buckets
room_buckets = (pd.read_csv('room_buckets.csv')
             .reset_index().set_index('bucket_size').to_dict()['num_rooms'])
room_buckets[0] = exams.shape[0]
room_buckets

{10: 7, 20: 41, 30: 20, 50: 13, 80: 8, 130: 1, 0: 230}

In the dictionary we just read, the keys are the set $R$, which makes things easier since the keys are also equal to the size of the room bucket, and the values are equal to $n_{rb}$. 
So `room_buckets[50] = 13`, means that we have $13$ rooms of size $50$ available. This will make things easier in our model, since instead of creating variables $x$ for every room, we want to create them for every room bucket, so the number of $x$ variables will drop a lot in size. Use the cell bellow to create the $x(i,rb,d,k)$ for each $i \in I, rb \in R, d \in P_i, k \in K$. Notice that now $x$ will denote the *number* of rooms of size $rb$ that we use for exam $i$, so it will be an integer variable instead of binary. Also notice that we have a room bucket of size $0$ and number of rooms the number of exams. A room of size zero might not seem useful, but we can use those rooms to schedule the online exams(recall we changed their enrollment to zero).

In [38]:
# define variables x here
x={}
x =  {(i,rb,d,k):m.IntVar(0, m.infinity(), 'x'+str([i,rb,d,k])) 
      for i in I for rb in room_buckets for d in exams.loc[i, ['prefdate','prefdate2','prefdate3']].tolist() 
      for k in range(K)}

Finally, for every exam, we will create a variable $z(i), \forall i \in I$, which is meant to store the total number of rooms used for that exam, which can be across different room buckets. While it might not be immediately clear why we need these variables, we will later use them to ensure that the total number of rooms used is not too large. 

In [39]:
# define variables z here
z={}
z =  {(i):m.IntVar(0, m.infinity(), 'z'+str([i])) 
      for i in I}

It is worth mentioning that since we are using room buckets now, a solution to the integer program we are creating will not give us both the day/time and rooms used for every exam. The idea is to use this program to find good day/time combinations, and ensure that we have enough rooms for each day/time by not using more than are avaible in the room buckets. After we have the exam to day/time assignment, we can then make a different integer program that will run on each slot and find "good" room assignments for the exams of that slot. We can use the objective function of that program to define what a "good" assignment satisfies. However for this lab, we only focus on finding a day/time assignment for the exams.

## Constraints

We can now start thinking about what constraints our model should have. For each constraint, we will first be describing what the constraint is meant to express/accomplish, then give the mathematical formulation of the constraint using the variables we have already defined, and finally you will have to code it in python.

#### $Z(i)$ and maximum number of rooms constraints

First remember that we wanted the variables $z(i)$ to correctly count the total number of rooms across all roombuckets assigned to an exam. To ensure that we want to create the appropriate constraint. We will also set an upper bound $R$ for the variables $z(i)$, that is limit the maximum number of rooms that an exam can be assigned to. So we have 
<center>
$z(i) = \sum\limits_{\substack{rb \in B\\ d \in P_i \\ k \in K}} x(i,rb,d,k),  \forall i \in I $ <br> <br>
 $z(i) \leq R, \forall i \in I$


In [44]:
# the z constraint
for i in I:
    m.Add(z[i] == sum(x[i,rb,d,k] 
                    for rb in room_buckets 
                    for d in exams.loc[i, ['prefdate','prefdate2','prefdate3']].tolist() 
                    for k in range(K)))
# code the maximum number of rooms constraints
R = 10 # defining max number of rooms R
for i in I:
    m.Add(z[i] <= R)

If you would like to check the size of your integer program as we add more constraints to it, you can use the `NumVariables()` and  `NumConstraints()` methods of solver to check the number of variables and constraints in the model.

In [45]:
print(f'Number of variables: {m.NumVariables()}')
print(f'Number of constraints: {m.NumConstraints()}')

Number of variables: 40710
Number of constraints: 1380


#### Prelim assigned unique slot constraint

Next we need a constraint to ensure that only $1$ of the $y$ variables corresponding to an exam can be non-zero, or in other words ensure that each exam is assigned to a unique day/time combination. <br><br>
<center>
 $\sum\limits_{\substack{d \in P_i \\ k \in K}} y(i,d,k) =1, \forall i \in I$

In [49]:
# code the unique slot constraint
for i in I:
    m.Add(1 == sum(y[i, d, k] for d in exams.loc[i, ['prefdate','prefdate2','prefdate3']].tolist() 
                    for k in range(K)))
         

#### Respecting the room bucket size for each slot

For every slot we do not want to go over the number of rooms that is available for a room bucket, 
<center>
    $\sum\limits_{i\in I:d \in P_i} x(i,rb,d,k) \le n_{rb}, \forall rb \in B, d \in D, k \in K$
<br> <br>
    
*Hint*: When you iterate over multiple sets, to make your code look cleaner, you can use `itertools.product()` as is done in the cell bellow.

In [56]:
# code the respecting room bucket size constraint
for rb,d,k in itertools.product(room_buckets, exam_dates, range(K)):
    m.Add(sum(x[i, rb, d, k] for i in I for d in exams.loc[i, ['prefdate','prefdate2','prefdate3']].tolist()) 
          <= room_buckets[rb])

#### Exams only using the room the day that it is assigned to them

We need a constraint to ensure that the $x$ variables can only have a non-zero value when the $y$ variable of the exam is $1$ <br><br>
<center>
    $ x(i,rb,d,k) \leq R y(i,d,k),\ \forall i \in I, d \in P_i, k \in K, rb \in B$

In [59]:
# code the room use on assigned day constraint
for i in I:
    for d in exams.loc[i, ['prefdate','prefdate2','prefdate3']].tolist():
        for k in range(K):
            for rb in room_buckets:
                m.Add(x[i, rb, d, k] <= R*y[i, d, k])

#### Exams must be given enough seats

Every exam should be giving enough seats for the number of students taking. <br><br>
<center>
$$
\sum_{\substack{rb \in B\\ d \in P_i \\ k \in K}} s_{rb} x(i,rb,d,k) \geq s_i, \forall i \in I$$

In [64]:
# code the exam seat constraint
for i in I:
    m.Add(sum((rb*room_buckets[rb])*x[i, rb, d, k] for rb in room_buckets 
              for d in exams.loc[i, ['prefdate','prefdate2','prefdate3']].tolist()
              for k in range(K)) >= exams.loc[i, ['enrollment']])

AttributeError: 'Series' object has no attribute 'AddSelfToCoeffMapOrStack'

#### Coenrollment conflict constraints

Recall the coenrollment dataframe we created before. 

In [ ]:
coenrollments

In this dataframe, we are given pairs of exams that have students in common. The coenrollment column has the number of students in common. Ideally we want to avoid scheduling exams at the same time, when there are a lot of students that need to take both exams. To accomplish that, we will add *coenrollment conflict* constraints to the model. We will create a variable for every pair of exams that appear in the dataframe, if they also have an overlap in their prefered days, as otherwise there would be no option to schedule them at the same time. We will use those variables to disincentivize scheduling the two exams at the same time, by adding a cost proportional to the number of students in common in the objective function.
We will be considering conflicts where we have more than $T$ students in common.
<br>
<br>
Suppose $CE$ is the set of pairs of exams that have more than $T$ students in common. So for any $(i,i') \in CE$ we want to create a binary variable $c(i,i')$, and if $i,i'$ also have an overlap in their prefered days, add a constraint to make $c(i,i')$ equal to $1$ if the exams $i,i'$ were scheduled at the same day/slot. This will be ensured by the following constraint:   <br><br>
<center>
$$
y(i,d,k) + y(i',d,k) -1 \le c(i,i'),\ (i,i')\in CE, d \in P_i \cap P_{i'}, k \in K
$$

*Hint:* You might find the python `set` object and the [`intersection()`](https://docs.python.org/2/library/sets.html#set-objects) method useful.

In [ ]:
# code the coenrollment conflicts constraints here
T = 2
# creating the set CE that was described above
tmp = coenrollments[coenrollments.coenrollment >= T][['exam_id_1','exam_id_2']]
CE = []
for index,row in tmp.iterrows():
    CE.append((row.exam_id_1,row.exam_id_2))

# defining c variables 
c =  {(i,i_prime):m.IntVar(0, 1, f'c{i,i_prime}') 
      for i,i_prime in CE}

# add the coenrollment conflict constraints 
raise ValueError('Replace this line with your code answer')

## Objective function

Before solving the model, the last thing we have to do is add the objective function. There are some things we are interested in minimizing in this model, and that should be accounted for in the objective function.
1. The total number of rooms used, across all prelims. 
2. We would like for as many classes as possible to get their first prefered date, and if not then their second one.
3. The total number of students that end up being involved in coenrollment conflicts.

  For 1. we can include the summation of the $z$ variables in the objective function to try to keep the total number of rooms to a minimum. So one term in the objective function will be <br>
  
  $$ \sum_{i\in I} z(i) $$
  
  
  For 2. we will use the $y$ variables in the objective function. Because ideally every class would get their first choice, assigning a class to their first choice should not incur a penalty in the objective function. Since we prefer the second choice over the third, the penalty for the second one should be smaller than the one for the third one. So we assign $wd_2 = 2$ and $wd_3 = 3$ as the weights for the variables associated with the second and third choice respectively. Then the term in the objective function will be: <br><br>
$$ \sum_{\substack{i \in I\\ k \in K}}wd_2 y(i,p_{i,2},k) +  \sum_{\substack{i \in I\\ k \in K}}wd_3 y(i,p_{i,3},k)$$  
   Finally for 3. we will use the $c$ variables, as they were created to indicate when a coenrollment conflict occurs. As for the weights for the $c$ variables $wc$, we use the dataframe to assign weight equal to the number of students in common minus $T$, so we get the number of students over the threshold that are involved in the conflict. So we get: 
      <br><br>
$$\sum_{\substack{i,i' \in C\\}}wc_{i,i'} c(i,i')$$

In [ ]:
# add the 3 terms described above to the objective function
# one term for the sumation of the z variables
# one term for the weighted number of classes that got their 2nd or 3rd preference
# one term to account for the number of students with coenrollment conflicts
wd_2 = 2
wd_3 = 3
m.Minimize(sum(z[i] for i in I) +
    sum(wd_2 * y[i,exams.loc[i, ['prefdate2']][0],k] for i in I for k in range(K)) + 
    sum(wd_3 * y[i,exams.loc[i, ['prefdate3']][0],k] for i in I for k in range(K)) +
    sum((coenrollments[(coenrollments.exam_id_1 == i)&
        (coenrollments.exam_id_2 == i_prime)].coenrollment.item()-T)*c[i,i_prime] 
        for i,i_prime in CE)
)

We can now solve the model and recover the assignment of exams to days/slots.

In [ ]:
### the next 3 lines are changing what the stopping point is for the IP solver
# instead of stopping when an optimal solution is found, now it will stop at a solution that is 
# withing 5% of the best bound at any given point
gap = 0.05
solverParams = pywraplp.MPSolverParameters()
solverParams.SetDoubleParam(pywraplp.MPSolverParameters().RELATIVE_MIP_GAP, gap)
# this line changes the maximum amount of time the model will run
# if an optimal solution has not been found by then, it returns the current best solution
# in the interest of saving time, we allow the model to run for only 2 minutes
m.set_time_limit(60*1000) 
m.Solve(solverParams)
print('Objective =',m.Objective().Value())
print(f'Solution:')
for v in m.variables():
    if v.solution_value() > 0:
        print(v.name(), v.solution_value())

We can see all the values of the variables, but that alone is not helpful. We would like to find what days/times the exams got assigned to, since that was what this model was meant to solve. To do so we will look at the $y$ variables that have value 1, and then store their indicies.

In [ ]:
# stores the indices of y variables that have value 1 in a list called schedule
from ast import literal_eval
schedule = [literal_eval(v.name()[1:]) for v in m.variables() if v.name()[0] == 'y' and v.solution_value() == 1]
schedule_df = pd.DataFrame(schedule, columns = ['exam_id', 'date', 'slot'])
schedule_df

This gives us an assignment of exams to date and slot.